# Learning a Communication Channel

Normally, if you have a function you would like to compute
across a connection, you would specify it with `function=my_func`
in the `FpgaPesEnsembleNetwork` constructor.
However, it is also possible to use error-driven learning
to learn to compute a function online.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import nengo
from nengo.processes import WhiteSignal

import nengo_fpga
from nengo_fpga.networks import FpgaPesEnsembleNetwork

## Step 1: Choose a Device

Select the FPGA device on which you wish to run the remote
`FpgaPesEnsembleNetwork`. This name corresponds with the name
in your `fpga_config` file. Recall the device name
is the name in square brackets which precedes the
settings, for example, **[de1]** or **[pynq]**.

In [ ]:
board = 'de1'  # Change this to your desired device name

## Step 2: Create the Network Without Learning

We'll start by creating a connection between two populations
that initially computes a very weird function.

In [ ]:
# Create a 'model' object to which we can add
# ensembles, connections, etc.
model = nengo.Network(label="Learned Comm Channel")
with model:
    # Arbitrary reference signal
    input_node = nengo.Node(WhiteSignal(60, high=5), size_out=1)
    
    # FPGA neural ensemble
    # (note that the learning_rate here does nothing
    # without an error signal)
    fpga_ens = FpgaPesEnsembleNetwork(
        board, n_neurons=60, dimensions=1, learning_rate=5e-5,
        label="FPGA ensemble",
        function=lambda x: np.random.random(1))
    
    # Connect the input to the FPGA ensemble
    # (the FPGA input is automatically passed to the output,
    # no need for an explicit connection)
    nengo.Connection(input_node, fpga_ens.input)
    
    # Add probes
    input_p = nengo.Probe(input_node, synapse=0.01)
    output_p = nengo.Probe(fpga_ens.output, synapse=0.01)

If we run this model as is, we can see that the FPGA ensemble does not compute a useful value.

In [ ]:
with nengo_fpga.Simulator(model) as sim:
    sim.run(10)

In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(sim.trange(), sim.data[input_p], c='k', label='Input')
plt.plot(sim.trange(), sim.data[output_p], c='r',
         label='FPGA Output')
plt.ylim(-1.5, 1.5)
plt.legend(loc='best')
plt.xlabel("Sim time")
plt.title("Random Initialization Without Learning");

## Step 3: Add Learning

If we can generate an error signal, then we can minimize
that error signal using the `nengo.PES` learning rule
that is built into the `FpgaPesEnsembleNetwork`.
Since it's a communication channel, we know the value that we want,
so we can compute the error with another ensemble.

In [ ]:
with model:
    # Ensemble to calculate error
    error = nengo.Ensemble(60, dimensions=1)
    error_p = nengo.Probe(error, synapse=0.03)

    # Compute the error (error = actual - target = post - pre)
    # In this case we are learning the square of the input
    nengo.Connection(fpga_ens.output, error)
    nengo.Connection(input_node, error, transform=-1)

    # Project error to the adaptive neural ensemble on the FPGA
    nengo.Connection(error, fpga_ens.error)

Now, we can see that the adaptive FPGA ensemble gradually learns the communication channel.

In [ ]:
with nengo_fpga.Simulator(model) as sim:
    sim.run(10)

In [ ]:
plt.figure(figsize=(16, 9))
plt.subplot(2, 1, 1)
plt.plot(sim.trange(), sim.data[input_p], c='k', label='Input')
plt.plot(sim.trange(), sim.data[output_p], c='r',
         label='FPGA Output')
plt.ylim(-1.5, 1.5)
plt.legend(loc='best')
plt.title("Random Initialization With Learning")
plt.subplot(2, 1, 2)
plt.plot(sim.trange(), sim.data[error_p], c='b', label="Error")
plt.ylim(-1, 1)
plt.xlabel("Sim time")
plt.legend(loc='best');

## Does it generalize?

If the learning rule is always working,
the error will continue to be minimized.
But have we actually generalized
to be able to compute the communication channel
without this error signal?
Let's inhibit the `error` population after 10 seconds.
We are forcing the neurons in the `error`
population to output zero.

In [ ]:
def inhibit(t):
    return 2.0 if t > 10.0 else 0.0

with model:
    inhib = nengo.Node(inhibit)
    nengo.Connection(inhib, error.neurons,
                     transform=[[-1]] * error.n_neurons)

In [ ]:
with nengo_fpga.Simulator(model) as sim:
    sim.run(16)

In [ ]:
plt.figure(figsize=(16, 9))
plt.subplot(2, 1, 1)
plt.plot(sim.trange(), sim.data[input_p], c='k', label='Input')
plt.plot(sim.trange(), sim.data[output_p], c='r',
         label='FPGA Output')
plt.ylim(-1.5, 1.5)
plt.legend(loc='best')
plt.title("Random Initialization With Learning")
plt.subplot(2, 1, 2)
plt.plot(sim.trange(), sim.data[error_p], c='b', label="Error")
plt.ylim(-1, 1)
plt.xlabel("Sim time")
plt.legend(loc='best');

## How Does This Work?

Since many of the internal dynamics of the FPGA ensemble
are not probable (for performance's sake), it is difficult
to explore the details of the implementation here.
Take a look at the [learn_communication_channel](https://www.nengo.ai/nengo/examples/learning/learn_communication_channel.html#How-does-this-work?)
example built with standard Nengo for an explanation of the
learning rule and how the connection weights change.